In [129]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [130]:
print("hello world")

hello world


In [131]:
ratings = pd.read_csv("../data/raw/ml-latest-small/ratings.csv")

In [132]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [133]:
movies = pd.read_csv("../data/raw/ml-latest-small/movies.csv")

In [134]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [135]:
tags = pd.read_csv("../data/raw/ml-latest-small/tags.csv")

In [136]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [137]:
tags_dict = tags.groupby("movieId")["tag"].apply(list).to_dict()

In [138]:
mv_data = ratings.merge(movies, on="movieId")

In [139]:
mv_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [140]:
user_matrix = ratings.pivot(index = ["userId"], columns = ["movieId"], values = "rating").fillna(0)

In [141]:
user_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
knn_model = NearestNeighbors(metric = "cosine", algorithm = "brute", n_neighbors = 10, n_jobs = -1)

In [143]:
knn_model.fit(user_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [149]:
def recommendation(mv_name, user_matrix, model, n_recommendations):
    model.fit(user_matrix)
    mv_id = movies.loc[movies["title"].str.contains(mv_name, case=False)]["movieId"].values[0]
    existing_users = user_matrix[user_matrix[mv_id] > 0].index
    existing_ratings = user_matrix.loc[existing_users]
    distances, indicies = model.kneighbors(existing_ratings, n_neighbors = n_recommendations)
    mv_recs = sorted(list(zip(indicies.squeeze().tolist(), distances.squeeze().tolist())), key = lambda x : x[1])[:0:-1]
    res = list()
    for mv in mv_recs:
        res.append({"Title": movies["title"][mv[0]], "distance": mv[1]})
    df = pd.DataFrame(res)[:n_recommendations]
    df.index += 1
    return df

In [183]:
df = recommendation("Inception", user_matrix, knn_model, 10)

In [185]:
df["Title"].iloc[0]

580                             Flipper (1996)
299                    Double Happiness (1994)
232                         Love Affair (1994)
600                        Stupids, The (1996)
486                        True Romance (1993)
246    New York Cop (Nyû Yôku no koppu) (1993)
470                              Sirens (1994)
416                     Jimmy Hollywood (1994)
257                        Pulp Fiction (1994)
188                        Castle Freak (1995)
Name: title, dtype: object

,Title,distance
1,468 Short C...,"[7.771561172376096e-16, 0.4986839063938976, 0...."
2,379 Coneheads (1993) 273 ...,"[6.661338147750939e-16, 0.5021351411438817, 0...."
3,136 Desperado (1995) 589 La...,"[5.551115123125783e-16, 0.6261461097528171, 0...."
4,18 Ace Ventura: When Nature Ca...,"[5.551115123125783e-16, 0.5645837460111773, 0...."
5,287 Star Trek: Generati...,"[5.551115123125783e-16, 0.5076089453878281, 0...."
